# Airline Passenger Satisfaction
Elimelech Berlin  
October 2023

## Overview

## Data

## EDA

In [32]:
import zipfile
import pandas as pd
import seaborn as sns

In [3]:
# open zipped file & create a dataframe for ech of the train/test subsets of the data
with zipfile.ZipFile('data/archive.zip', 'r') as zipped_file:
    for file in zipped_file.namelist():
        file_csv = zipped_file.open(file)
        file_df = pd.read_csv(file_csv)
        if file == 'test.csv':
            test_df = file_df
        if file == 'train.csv':
            train_df = file_df

In [33]:
print('# of rows in the train set: {}'.format(len(train_df)))
print('# of rows in the test set: {}'.format(len(test_df)))
print('This is equal to a train/test split of {}/{}'.format(int(len(train_df) / (len(train_df) + len(test_df)) * 100), int(len(test_df) / (len(train_df) + len(test_df)) * 100)))

# of rows in the train set: 103904
# of rows in the test set: 25976
This is equal to a train/test split of 80/20


To gain understanding of the dataset, have a look at the column names:

In [34]:
train_df.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

Accurding to the written down stuff about the data provided with the data, some of the columns contain subjective information provided by consumers surveyed about their past experiences. Such information may not be available when predicting overall satisfaction prior to a customer's survey response & should therefore be excluded from any model employed to predict a rating in order to bribe a customer at high risk of dissatisfaction in order to change his/her mind. (If the point of a model is too know who will be dissatisfied, in order to take measures to preemptively bribe them, we can't use as of yet unprovided ratings on 'food & drink service' to predict that info. A solution to this can be to request ratings for different elements of consumer experience as the service is provided. i.e. customers are asked to rate ease-of-online booking service immediately upon completion of booking, not as part of an overall survey.)